This notebooks shows how we have implemented ngmix v1.3.9 in SuperBIT pipeline.  

Similar piece of code has been implemented in [```superbit_lensing/metacalibration/ngmix_fit_superbit3.py```](https://github.com/superbit-collaboration/superbit-lensing/blob/main/superbit_lensing/metacalibration/ngmix_fit_superbit3.py#L438-L495)

In [1]:
import numpy as np
import ngmix
from ngmix import joint_prior
from ngmix.priors import *
import galsim
from astropy.table import Table, Row, vstack, hstack
from ngmix.medsreaders import NGMixMEDS
from copy import deepcopy
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
norm=colors.Normalize()
norm2 = colors.SymLogNorm(linthresh=1e-4,base=np.e)
norm3 = colors.SymLogNorm(linthresh=1e-3,base=np.e)

In [2]:
seed = 42
rng = np.random.RandomState(seed)
fname = '/work/mccleary_group/saha/data/PLCKG287d0p32d9/b/backups/PLCKG287d0p32d9_b_meds.fits'
medsObj = NGMixMEDS(fname)

In [3]:
def _get_priors(seed):

    # This bit is needed for ngmix v2.x.x
    # won't work for v1.x.x
    rng = np.random.RandomState(seed)

    # prior on ellipticity.  The details don't matter, as long
    # as it regularizes the fit.  This one is from Bernstein & Armstrong 2014

    g_sigma = 0.3
    g_prior = ngmix.priors.GPriorBA(g_sigma, rng=rng)

    # 2-d gaussian prior on the center
    # row and column center (relative to the center of the jacobian, which would be zero)
    # and the sigma of the gaussians

    # units same as jacobian, probably arcsec
    row, col = 0.0, 0.0
    row_sigma, col_sigma = 0.2, 0.2 # a bit smaller than pix size of SuperBIT
    cen_prior = ngmix.priors.CenPrior(row, col, row_sigma, col_sigma, rng=rng)

    # T prior.  This one is flat, but another uninformative you might
    # try is the two-sided error function (TwoSidedErf)

    Tminval = -1.0 # arcsec squared
    Tmaxval = 1000
    T_prior = ngmix.priors.FlatPrior(Tminval, Tmaxval, rng=rng)

    # similar for flux.  Make sure the bounds make sense for
    # your images

    Fminval = -1.e1
    Fmaxval = 1.e5
    F_prior = ngmix.priors.FlatPrior(Fminval, Fmaxval, rng=rng)

    # now make a joint prior.  This one takes priors
    # for each parameter separately
    priors = ngmix.joint_prior.PriorSimpleSep(
    cen_prior,
    g_prior,
    T_prior,
    F_prior)

    return priors

def make_struct(res, shear_type):
    """
    make the data structure

    Parameters
    ----------
    res: dict
        With keys 's2n', 'e', and 'T'
    obs: ngmix.Observation
        The observation for this shear type
    shear_type: str
        The shear type

    Returns
    -------
    1-element array with fields
    """
    dt = [
        ('flags', 'i4'),
        ('shear_type', 'U7'),
        ('s2n', 'f8'),
        ('g', 'f8', 2),
        ('T', 'f8'),
        ('Tpsf', 'f8'),
    ]
    data = np.zeros(1, dtype=dt)
    data['shear_type'] = shear_type
    data['flags'] = res['flags']

    if res['flags'] == 0:
        data['s2n'] = res['s2n']
        # for Gaussian moments we are actually measureing e, the ellipticity
        try:
            data['g'] = res['e']
        except KeyError:
            data['g'] = res['g']
        data['T'] = res['T']
        data['Tpsf'] = res['Tpsf']
    else:
        data['s2n'] = np.nan
        data['g'] = np.nan
        data['T'] = np.nan
        data['Tpsf'] = np.nan

    return data

In [4]:
obs = medsObj.get_obslist(4444, weight_type='uberseg')
se_obslist = ngmix.ObsList(meta=deepcopy(obs._meta))
for ob in obs[1:]: # to exclude meds coadd
    se_obslist.append(ob)
obslist = se_obslist

In [5]:
mcal_shear = 0.01
lm_pars = {'maxfev':2000, 'xtol':5.0e-5, 'ftol':5.0e-5}
metacal_pars={'step':mcal_shear, 'rng': rng}
max_pars = {'method':'lm', 'lm_pars':lm_pars, 'find_center':True}
prior = _get_priors(seed)
# get image pixel scale (assumes constant across list)
jacobian = obslist[0]._jacobian
Tguess = 4*jacobian.get_scale()**2
ntry = 20
psf_model = 'coellip5'
gal_model = 'gauss'

# Run the actual metacalibration fits on the observed galaxies
mcb = ngmix.bootstrap.MaxMetacalBootstrapper(obslist)
mcb.fit_metacal(psf_model, gal_model, max_pars, Tguess, prior=prior,
                ntry=ntry, metacal_pars=metacal_pars)
resdict = mcb.get_metacal_result() 

/work/mccleary_group/saha/codes/ngmix-1.3.9/ngmix/fitting_nb.py:169: NumbaExperimentalFeatureWarning: Record(u[type=float64;offset=0],v[type=float64;offset=8],val[type=float64;offset=16],ierr[type=float64;offset=24],fdiff[type=float64;offset=32];40;False) has been considered a subtype of Record(u[type=float64;offset=0],v[type=float64;offset=8];16;False)  This is an experimental feature.
  model_val = gmix_eval_pixel_fast(gmix, pixel)


In [6]:
dlist = []
for stype, sres in resdict.items():
    try: 
        st = make_struct(res=sres, shear_type=stype)
    except Exception as e:
        print(e)
        continue
    dlist.append(st)

data = np.hstack(dlist)

'int' object is not subscriptable


In [7]:
data 

array([(0, '1m', 90.48326187, [ 0.11351608, -0.23308908], 0.11065499, 1.31885876),
       (0, '1m_psf', 90.99163723, [ 0.14853564, -0.23175385], 0.10345196, 1.21197285),
       (0, '1p', 90.44040051, [ 0.13086354, -0.22573577], 0.11161491, 1.31175512),
       (0, '1p_psf', 90.36487992, [ 0.10239476, -0.22800108], 0.12257008, 1.27122   ),
       (0, '2m', 90.62756257, [ 0.13265692, -0.25356834], 0.10742093, 1.27322336),
       (0, '2m_psf', 90.72031685, [ 0.12029177, -0.23504861], 0.10308314, 1.32912777),
       (0, '2p', 90.84804008, [ 0.14398122, -0.24638765], 0.09643692, 1.31893382),
       (0, '2p_psf', 91.75178873, [ 0.00921386, -0.22595474], 0.17296009, 1.17149645),
       (0, 'noshear', 90.52408907, [ 0.10989519, -0.23251989], 0.10186874, 1.29902004)],
      dtype=[('flags', '<i4'), ('shear_type', '<U7'), ('s2n', '<f8'), ('g', '<f8', (2,)), ('T', '<f8'), ('Tpsf', '<f8')])